In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy.integrate import solve_ivp
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from tqdm import tqdm

In [2]:
# Regular Pendulum Dynamics
def regular_pendulum(t, y, l, g):
    theta, z = y
    dydt = np.zeros_like(y)
    dydt[0] = z
    dydt[1] = -(g / l) * np.sin(theta)
    return dydt

In [3]:
# Main Workflow
n_pendulums = 100
output_dir = "pendulum_video_dataset"

In [4]:
# Generate Dataset
def generate_dataset(n_pendulums, dt, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    dataset = []

    for i in tqdm(range(n_pendulums), desc="Generating dataset"):
        l = np.random.uniform(0.5, 2.0)
        g = 9.81
        y0 = np.random.uniform(-np.pi, np.pi, 2)
        t_span = (0, 10)
        t_eval = np.linspace(t_span[0], t_span[1], int(10 / dt))

        sol = solve_ivp(regular_pendulum, t_span, y0, t_eval=t_eval, args=(l, g), method='RK45')
        data = sol.y[0]

        gif_path = os.path.join(output_dir, f"pendulum_{i}.gif")
        generate_gif(data, l, gif_path)

        dataset.append((data, l, g, gif_path))

    return dataset

In [5]:
# Generate GIFs
def generate_gif(data, l, save_path):
    theta = data
    x = l * np.sin(theta)
    y = -l * np.cos(theta)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(-2.5, 2.5)
    ax.set_ylim(-2.5, 2.5)
    ax.set_aspect('equal')
    line, = ax.plot([], [], 'o-', lw=2)

    def update(frame):
        line.set_data([0, x[frame]], [0, y[frame]])
        return line,

    ani = FuncAnimation(fig, update, frames=len(theta), blit=True, interval=50)
    ani.save(save_path, fps=20, writer='imagemagick')
    plt.close(fig)

In [6]:
dataset = generate_dataset(n_pendulums, dt=0.01, output_dir=output_dir)

Generating dataset: 100%|██████████| 100/100 [1:02:00<00:00, 37.20s/it]


In [7]:
# Dataset Preparation
class PendulumDataset(Dataset):
    def __init__(self, dataset, seq_len, pred_len):
        self.dataset = dataset
        self.seq_len = seq_len
        self.pred_len = pred_len

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        data, l, g, gif_path = self.dataset[idx]

        gif = Image.open(gif_path)
        frames = []
        try:
            while True:
                frame = gif.convert("RGB")
                frames.append(np.array(frame))
                gif.seek(gif.tell() + 1)
        except EOFError:
            pass

        positions = np.column_stack([data])

        input_seq = positions[:self.seq_len]
        target_seq = positions[self.seq_len:self.seq_len + self.pred_len]

        return torch.tensor(input_seq, dtype=torch.float32), torch.tensor(target_seq, dtype=torch.float32), gif_path

In [8]:
seq_len = 90
pred_len = 10
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_dataset = PendulumDataset(train_data, seq_len, pred_len)
val_dataset = PendulumDataset(val_data, seq_len, pred_len)
test_dataset = PendulumDataset(test_data, seq_len, pred_len)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [9]:
input_dim = 1
hidden_dim = 64
output_dim = 1
num_layers = 2

In [10]:
# Model Definition
class LSTMPendulumPredictor(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers):
        super(LSTMPendulumPredictor, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out.unsqueeze(1)

In [11]:
# Training the Model
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        for inputs, targets, _ in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for inputs, targets, _ in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}")

In [12]:
# Generate Prediction GIFs
def generate_prediction_gif(model, test_loader, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model.eval()

    for i, (inputs, targets, gif_path) in enumerate(test_loader):
        with torch.no_grad():
            predictions = model(inputs).squeeze(0).numpy()

        true_positions = targets.numpy()
        input_positions = inputs.squeeze(0).numpy()

        fig, ax = plt.subplots(figsize=(6, 6))
        ax.set_xlim(-2.5, 2.5)
        ax.set_ylim(-2.5, 2.5)
        ax.set_aspect('equal')
        true_line, = ax.plot([], [], 'o-', lw=2, label='True Motion')
        pred_line, = ax.plot([], [], 'o-', lw=2, linestyle='--', label='Predicted Motion')

        def update(frame):
            if frame < len(input_positions):
                true_line.set_data([0, input_positions[frame]],
                                   [0, input_positions[frame]])
            else:
                idx = frame - len(input_positions)
                true_line.set_data([0, true_positions[idx]],
                                   [0, true_positions[idx]])
                pred_line.set_data([0, predictions[idx]],
                                   [0, predictions[idx]])
            return true_line, pred_line

        ani = FuncAnimation(fig, update, frames=len(input_positions) + len(true_positions), blit=True, interval=50)
        ani.save(os.path.join(output_dir, f"prediction_{i}.gif"), fps=20, writer='imagemagick')
        plt.close(fig)

In [13]:
model = LSTMPendulumPredictor(input_dim, hidden_dim, output_dim, num_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [14]:
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1/20:   0%|          | 0/3 [00:00<?, ?it/s]C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32, 10, 1])) that is different to the input size (torch.Size([32, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch 1/20:  67%|██████▋   | 2/3 [02:56<01:27, 87.83s/it]C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([16, 10, 1])) that is different to the input size (torch.Size([16, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self

Epoch 1, Train Loss: 3.1426893870035806, Val Loss: 6.685717582702637


Epoch 2/20: 100%|██████████| 3/3 [03:43<00:00, 74.65s/it]


Epoch 2, Train Loss: 2.8521271546681723, Val Loss: 6.368018627166748


Epoch 3/20: 100%|██████████| 3/3 [03:44<00:00, 74.67s/it]


...

Epoch 5, Train Loss: 2.578622261683146, Val Loss: 4.793635845184326


Epoch 6/20: 100%|██████████| 3/3 [03:48<00:00, 76.01s/it]


Epoch 6, Train Loss: 1.7274532318115234, Val Loss: 4.036993503570557


Epoch 7/20:  33%|███▎      | 1/3 [01:42<03:24, 102.49s/it]


KeyboardInterrupt: 

In [ ]:
generate_prediction_gif(model, test_loader, output_dir="predicted_gifs")